In [1]:
import os
import pandas as pd
from google.cloud import bigquery
from google.cloud import storage
from google.oauth2.service_account import Credentials

In [2]:
# BIGQUERY_CREDENTIALS_FILE_PATH = r"D:\data_engineer\TIR_group2\TIR101_Group2\secrets\harry_GCS_BigQuery_write_cred.json"
BIGQUERY_CREDENTIALS_FILE_PATH = r"C:\dev_TIR101\Taipei-transit-data_hub\airflow\dags\harry_GCS_BigQuery_write_cred.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = BIGQUERY_CREDENTIALS_FILE_PATH
BQ_CLIENT = bigquery.Client()
GCS_CLIENT = storage.Client()

In [3]:
def cteate_MRT_history_external_table(dataset_name:str,table_name:str,client:bigquery.Client=BQ_CLIENT):
    query_job = client.query(
        f"""
    CREATE OR REPLACE EXTERNAL TABLE `{dataset_name}.{table_name}`
    (
        `date` DATE,
        `hour` INT ,
        `mrt_station_name_enter` STRING,
        `mrt_station_name_exit` STRING,
        `visitors_num` INT,
    )
    OPTIONS (
        format = 'CSV',
        uris = ['gs://mrt_history_usage/*.csv'],
        skip_leading_rows = 1,
        max_bad_records = 1,
        encoding ='UTF-8'
    );
    """
    )
    query_job.result()
    print(f"{dataset_name}.{table_name} has been created")
cteate_MRT_history_external_table(dataset_name="MRT_history",table_name="SRC_MRT_history_usage",client=BQ_CLIENT)

MRT_history.SRC_MRT_history_usage has been created


In [5]:
def cteate_MRT_history_ODS_table(dataset_name:str,create_table_name:str,SRC_table_name:str,client:bigquery.Client=BQ_CLIENT):
    query_job = client.query(
    f"""
    CREATE OR REPLACE TABLE `{dataset_name}.{create_table_name}` AS
    SELECT 
        date ,
        hour ,
        REGEXP_REPLACE(mrt_station_name_enter ,'[A-Za-z]','') AS mrt_station_name_enter,
        REGEXP_REPLACE(mrt_station_name_exit ,'[A-Za-z]','') AS mrt_station_name_exit,
        visitors_num,
    FROM `{dataset_name}.{SRC_table_name}`
    ;
    """
    )
    query_job.result()
    print(f"{dataset_name}.{create_table_name} has been created")
cteate_MRT_history_ODS_table(dataset_name="MRT_history",create_table_name="ODS_MRT_history_usage",SRC_table_name="SRC_MRT_history_usage",client=BQ_CLIENT)

MRT_history.ODS_MRT_history_usage has been created


In [ ]:
    # pattern = re.compile(r"[A-Za-z]+")
    # df["進站"] = df["進站"].str.replace(pattern, "", regex=True)
    # df["出站"] = df["出站"].str.replace(pattern, "", regex=True)
    # df.rename(columns={
    #     "日期": "date",
    #     "時段": "hour",
    #     "進站": "mrt_station_name_enter",
    #     "出站": "mrt_station_name_exit",
    #     "人次": "visitors_num"
    # }, inplace=True)
    # print(f"T_mrt_usage_history finished")